# This notebook provides portfolio weights based on the historical data to the backtesting notebook

In [1]:
# !pip uninstall deepdowmine
#!pip install git+https://github.com/dsman1823/deepdowmine.git
#!conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch # <---- to Enable SVD 

In [2]:
import torch

import pandas as pd
import numpy as np

from datetime import datetime 
from deepdowmine.nn import LinearNetMine, UpdNumericalMarkowitzWithShorting, UpdLinearNetMine, UpdDenseNet, UpdUpdLinearNetMine
from deepdowmine.nn import DenseNetMinVar, ConvNetFullOpti, ConvNetMinVar, RnnNetMinVar, LstmNetMinVar, RnnNetFullOpti
from deepdowmine.nn import LstmNetFullOpti

In [3]:
RETS_FILE_PATH = 'historical_returns.csv'



loockback, gap, horizon = 50, 0, 5
n_assets = 5
loockback = 50


## Load NN from dict

In [4]:
LINEAR_NET_OSHARPE_FILE_PATH = r'./NNs/linear_net_50x5_sharpe.pth' # LinearNetMine with ordinary Sharpe cost

In [5]:
# years parameter reprsenets amount of years after the backtest stasrt: the model dowloaded is the the model trained 
# on the period (train_start, back_test_start + years) 


# LinearNetMine with ordinary sharpe cost function
def load_linear_net_osharpe(years=0):
    network = LinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{years}.pth'))
    print(fr'./NNs/linear_net_50x5_sharpe_{years}.pth')
    return network.eval()
    

In [6]:
def load_dense_net(years=0):
    network = UpdLinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/dense_{years}.pth'))
    print(fr'dense_{years}.pth')
    return network.eval()

## Core functions

In [7]:
def transform_rets_to_NN_input(rets):
    # tranform (loockback, n_assets) df into (1, 1, loockback, n_assets)=(sample_size, n_channels, loockback, n_assets) tensor 
    returns_np = rets.to_numpy()

    # Add the required dimensions: (n_samples, n_channels, lookback, n_assets)
    returns_np_expanded = np.expand_dims(returns_np, axis=0)  # Adds n_samples dimension
    returns_np_expanded = np.expand_dims(returns_np_expanded, axis=0)  
    return torch.from_numpy(returns_np_expanded).float()

def weights_from_NN(rets, network):
    X = transform_rets_to_NN_input(rets)
    return network(X).detach().numpy()[0]

# method is a function, used for obtaining the returns
def get_weights(method):
    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    return method(rets)

In [8]:
rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
X = transform_rets_to_NN_input(rets)
network = LinearNetMine(1, loockback, n_assets, p=0.5)
network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{2}.pth'))
network.eval()(X).detach().numpy()[0]

array([ 0.06418574,  0.25817055, -0.01574563,  0.60880786,  0.08458146],
      dtype=float32)

## NNs and related methods


### NNs

networks[i] method is trained on (train_start, backtest_start + i years) data interval

In [9]:
years = [0, 1, 2]#, 1, 2]

linear_osharpe_networks = [load_linear_net_osharpe(years=i) for i in years]
dense_networks = [load_dense_net(years=i) for i in years]

./NNs/linear_net_50x5_sharpe_0.pth
./NNs/linear_net_50x5_sharpe_1.pth
./NNs/linear_net_50x5_sharpe_2.pth
dense_0.pth
dense_1.pth
dense_2.pth


### methods

In [10]:
dense_networks[1].linear0.bias[0:3]

tensor([ 0.0484,  0.0029, -0.0442], grad_fn=<SliceBackward0>)

In [11]:
dense_networks[2].linear0.bias[0:3]

tensor([ 0.0484,  0.0029, -0.0442], grad_fn=<SliceBackward0>)

In [12]:
# note the format of lambda 
linear_osharpe_methods = [lambda r, i=i: weights_from_NN(r, linear_osharpe_networks[i]) for i in years] 
dense_methods = [lambda r, i=i: weights_from_NN(r, dense_networks[i]) for i in years] 


In [13]:
[get_weights(dense_methods[i]) for i in years]

[array([0.01819081, 0.24908274, 0.29753768, 0.41258937, 0.0225994 ],
       dtype=float32),
 array([-0.03791929,  0.1064674 ,  0.4017897 ,  0.46670547,  0.06295667],
       dtype=float32),
 array([-0.03175861,  0.18791872,  0.14897145,  0.48070765,  0.21416074],
       dtype=float32)]

# Server

In [14]:
from dateutil.relativedelta import relativedelta
from datetime import datetime, date

RETRAINING_DATE1 = date(2021, 12, 15)
RETRAINING_DATE2 = date(2022, 12, 15)

In [15]:
tmp = True
not tmp

False

In [16]:
from deepdowmine.nn import DenseNetFullOpti, ConvNetMinVar, DenseNetFullOpti

In [24]:
from flask import Flask, request, jsonify
from flask_cors import CORS 

import numpy as np

app = Flask(__name__)
CORS(app)  # Enable CORS for your Flask app

markers = {}
markers['net_initialized'] = False

@app.route('/get_weights', methods=['GET'])
def get_request():
    # Extract date string from the query parameters
    date_str = request.args.get('date', None)
    
    # Simple validation to check if date is provided
    if not date_str:
        return jsonify({"error": "Missing date parameter"}), 400

    # Try to convert the date string to a datetime object
    try:
        # Note that now we're only parsing the date, not the time
        date = datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        # If there is an error in parsing the date, return an error message
        return jsonify({"error": "Invalid date format. Please use YYYY-MM-DD format."}), 400
    
    years_plus = 0
    if RETRAINING_DATE1 < date < RETRAINING_DATE2:
        years_plus = 1
        print(f'Working with model: {years_plus}')
    if RETRAINING_DATE2 < date:
        years_plus = 2
        print(f'Working with model: {years_plus}')
    
    weights = get_weights(linear_osharpe_methods[years_plus]).tolist()
    
    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    X = transform_rets_to_NN_input(rets)
    
    network = DenseNetMinVar(1, 50, 5, .2)
    network.load_state_dict(torch.load(fr'./NNs/trained_on_new_data/dense/min_var_1.pth'))
       
#     if date < RETRAINING_DATE1:
#         print(0)
#     if RETRAINING_DATE1 < date < RETRAINING_DATE2:
#         print('1')
#         network = DenseNetMinVar(1, 50, 5, .2)
#         network.load_state_dict(torch.load(fr'./NNs/trained_on_new_data/dense/min_var_1.pth'))
  
#     if RETRAINING_DATE2 < date:
#         network = DenseNetMinVar(1, 50, 5, .2)
#         network.load_state_dict(torch.load(fr'./NNs/trained_on_new_data/dense/min_var_2.pth'))
#         print('2')
    
#     network = RnnNetMinVar(5)
#     network.load_state_dict(torch.load(fr'./NNs/rnn_minvar.pth'))

    
    weights = network.eval()(X).detach().numpy()[0].tolist()
    
        
    #weights = weights_from_NN(rets, linear_osharpe_networks[0]).tolist()
    #print(weights)
    # Return the vector as JSON, using the string representation of the date for simplicity
    print(weights)
    return jsonify({'weights': weights})


In [ ]:
app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Mar/2024 21:13:24] "GET /get_weights?date=2020-12-21 HTTP/1.1" 200 -


[-0.14521503448486328, 0.2829407751560211, 0.0478636771440506, 0.6555772423744202, 0.1588331162929535]


127.0.0.1 - - [24/Mar/2024 21:13:26] "GET /get_weights?date=2020-12-28 HTTP/1.1" 200 -


[-0.11531110852956772, 0.10289134085178375, 0.08838795870542526, 0.8214763402938843, 0.10255534946918488]


...

Working with model: 2
[-0.07245595753192902, 0.6143446564674377, 0.06540947407484055, 0.5237219929695129, -0.13102035224437714]


127.0.0.1 - - [24/Mar/2024 21:19:08] "GET /get_weights?date=2023-12-04 HTTP/1.1" 200 -


Working with model: 2
[-0.09374786913394928, 0.20376500487327576, 0.07681745290756226, 0.7930110096931458, 0.020154325291514397]


127.0.0.1 - - [24/Mar/2024 21:19:10] "GET /get_weights?date=2023-12-11 HTTP/1.1" 200 -


Working with model: 2
[0.18092943727970123, 0.35742270946502686, -0.0837646946310997, 0.6254655718803406, -0.08005308359861374]


In [18]:
update the 

SyntaxError: invalid syntax (1388808595.py, line 1)